In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Chaitanya\\Documents\\ML\\Indian Medical Leaf Clf\\Medicinal-Leaf-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Chaitanya\\Documents\\ML\\Indian Medical Leaf Clf\\Medicinal-Leaf-Classification'

### entity (config_entity.py)

In [5]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen = True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path 

### configuration (configuration.py)

In [6]:
from MedicineLeafClassifier.constants import *
from MedicineLeafClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH  ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:

        config = self.config.prepare_callbacks
            
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config

### Components (prepare_callbacks.py)

In [8]:
import os 
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbacksConfig):
        self.config = config

    
    @property
    def create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(self.config.checkpoint_model_filepath,"model_{epoch}.h5"),  # Replace with your desired path
            save_best_only=True,  # Set to True to save only the best model based on a metric
            monitor='val_accuracy',  # Monitor validation accuracy during training
            save_weights_only=False,  # Set to True to save only model weights
            verbose=1  # Set to 0 for silent operation
        )
    @property
    def create_early_stopping(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',  # Monitor validation accuracy
            min_delta=0.01,  # Minimum required change in the monitored metric
            patience=50,  # Number of epochs with no improvement to wait before stopping
            baseline = 0.5,
            restore_best_weights=True  # Restore the weights of the best model before stopping
        )
    @property
    def get_tb_ckpt_callbacks(self):
        return [
            self.create_tb_callback,
            self.create_ckpt_callbacks,
            self.create_early_stopping
        ]

### pipeline (pipeline is not required for callbacks --> called while training )

In [20]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks

except Exception as e:
    raise e

[2024-02-29 14:57:46,661: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-29 14:57:46,666: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-29 14:57:46,668: INFO: common: Created directory at artifacts]
[2024-02-29 14:57:46,669: INFO: common: Created directory at artifacts\prepare_callbacks\checkpoint_dir]
[2024-02-29 14:57:46,669: INFO: common: Created directory at artifacts\prepare_callbacks\tensorboard_log_dir]
